In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [3]:
for folder in os.listdir()[1:7]:
    print(os.listdir(folder))

[]
['CSR_Projects_2014_2015.csv', 'CSR_year_2014_2015.csv', 'master_data_2014_2015.csv', 'no_master_data_2014_2015.csv']
['CSR_Projects_2015_2016.csv', 'CSR_year_2015_2016.csv', 'master_data_2015_2016.csv', 'no_master_data_2015_2016.csv']
['CSR_Projects_2016_2017.csv', 'CSR_year_2016_2017.csv', 'master_data_2016_2017.csv', 'no_master_data_2016_2017.csv']
['CSR_Projects_2017_2018.csv', 'CSR_year_2017_2018.csv', 'master_data_2017_2018.csv', 'no_master_data_2017_2018.csv']
['CSR_Projects_2018_2019.csv', 'CSR_year_2018_2019.csv', 'master_data_2018_2019.csv', 'no_master_data_2018_2019.csv']


In [4]:
projects= pd.read_csv("2018-19\CSR_Projects_2018_2019.csv")
csr_18= pd.read_csv("2018-19\CSR_year_2018_2019.csv")
company_data= pd.read_csv('2018-19\master_data_2018_2019.csv')
no_master= pd.read_csv("2018-19/no_master_data_2018_2019.csv")

In [5]:
def clean_data(projects, csr, company_data, no_master, year):
    #Removing grand totals
    projects = projects[projects["S.No."] != "Grand Total"]
    projects = projects.drop(["S.No.", "Feedback"], axis=1)
    
    #merging master and no_master
    company_data = company_data.append(no_master,ignore_index=True)
    
    csr["year"]= year
    company_data = company_data.merge(csr.drop("Company Name", axis=1), left_on="CIN", right_on="CIN_Number").drop("CIN", axis=1)
    
    #merging company data and projects data
    master_data = projects.merge(company_data.drop(["year"], axis=1).rename(columns={"State":"Company_HQ"}),on="CIN_Number")
    master_data["year"]=year
    
    #rearranging the columns

    companay_name='company name'
    if companay_name not in master_data.columns:
        master_data = master_data.rename(columns={"Company Name":"company name"})
        
    master_data = master_data[[ 'CSR Project(s)',  'Project Amount Outlay', 'Amount Spent','year', 'Development Sector(s)', 
             'State', 'District', 'Mode of Implementation',companay_name, 'CIN_Number',  'Average Net Profit',
           'CSR Prescribed Expenditure', 'CSR Spent', 'Local Area Spent', 'Company_HQ',"RoC",  'Class', 'Company Type',  
            'Sub Category', 'Listing Status' ]]
    
    #removing some rows
    master_data = master_data[
                ((master_data["CSR Prescribed Expenditure"] !=0) | (master_data["CSR Spent"] !=0)) 
                     |   ((master_data["Project Amount Outlay"] != 0.0) | (master_data["Amount Spent"] != 0.0))
                ]
    
    master_data=master_data.rename(columns={ 
                 'State': "State_project", 'District':"District_project", "Company_HQ":'State_company',
    })

    master_data.to_csv("master_data_"+ year+ ".csv", index=False)


In [6]:
for folder in os.listdir()[2:7]:
    data_path = [folder+ "/"+ i for i in os.listdir(folder)]
        
    projects, csr, master, no_master = [pd.read_csv(path) for path in data_path ]
    
    clean_data(projects, csr, master, no_master, folder)

In [7]:
files = os.listdir()    
files = list(filter(lambda f: f.endswith('.csv'), files))
datasets= [pd.read_csv(file) for file in files[:-1]]

master_dataset = pd.concat(datasets, ignore_index=True)

In [68]:
master_dataset.groupby("year")["Amount Spent",'CSR Prescribed Expenditure',"CSR Spent", "Local Area Spent"].agg("sum")

<ipython-input-68-a1612afbe437>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  master_dataset.groupby("year")["Amount Spent",'CSR Prescribed Expenditure',"CSR Spent", "Local Area Spent"].agg("sum")


,Amount Spent,CSR Prescribed Expenditure,CSR Spent,Local Area Spent
year,,,,
2014-15,1.006593e+11,227753.744853,140324.050000,31135.800000
2015-16,1.451721e+11,253818.837503,234535.537496,70631.183930
2016-17,1.433223e+11,276442.980000,272555.390238,126717.750715
2017-18,1.370802e+11,175680.786273,169086.930000,87400.020034
2018-19,1.862805e+11,425832.799045,430163.130000,268657.169638


In [61]:
master_dataset.loc[master_dataset["CSR Prescribed Expenditure"].abs()>10000 ,["CSR Prescribed Expenditure"]]=master_dataset.loc[master_dataset["CSR Prescribed Expenditure"]>10000 ,["CSR Prescribed Expenditure"]]/10000000
master_dataset.loc[master_dataset['CSR Spent'].abs()>10000 ,['CSR Spent']]=master_dataset.loc[master_dataset['CSR Spent']>10000 ,['CSR Spent']]/10000000
master_dataset.loc[master_dataset['Average Net Profit'].abs()>10000 ,['Average Net Profit']]=master_dataset.loc[master_dataset['Average Net Profit']>10000 ,['Average Net Profit']]/10000000
master_dataset.loc[master_dataset['Local Area Spent'].abs()>10000 ,['Local Area Spent']]=master_dataset.loc[master_dataset['Local Area Spent']>10000 ,['Local Area Spent']]/10000000

master_dataset

,CSR Project(s),Project Amount Outlay,Amount Spent,year,Development Sector(s),State_project,District_project,Mode of Implementation,company name,CIN_Number,Average Net Profit,CSR Prescribed Expenditure,CSR Spent,Local Area Spent,State_company,RoC,Class,Company Type,Sub Category,Listing Status
0,NaN,0.00,0.0,2014-15,NEC/ Not Mentioned,NEC/ Not mentioned,NEC/ Not mentioned,NaN,SAB INDUSTRIES LIMITED,L00000CH1983PLC031318,0.00,0.148671,0.00,0.00,Chandigarh,RoC-Chandigarh,Public,Non-govt company,Company limited by Shares,Listed
1,roof water proofing,500000.00,0.0,2014-15,Environmental Sustainability,Madhya Pradesh,Hoshangabad,Directly,Natraj Proteins Limited,L00153MP1990PLC006090,11.38,0.210000,0.21,0.01,Madhya Pradesh,RoC-Gwalior,Public,Other than Govt.,Company limited by shares,Listed
2,training to the poor classes of people fro emp...,800000.00,0.0,2014-15,Special Education,Madhya Pradesh,Hoshangabad,Directly,Natraj Proteins Limited,L00153MP1990PLC006090,11.38,0.210000,0.21,0.01,Madhya Pradesh,RoC-Gwalior,Public,Other than Govt.,Company limited by shares,Listed
3,education society,500000.00,0.0,2014-15,Education,Madhya Pradesh,Hoshangabad,Directly,Natraj Proteins Limited,L00153MP1990PLC006090,11.38,0.210000,0.21,0.01,Madhya Pradesh,RoC-Gwalior,Public,Other than Govt.,Company limited by shares,Listed
4,public waiting hall,500000.00,0.0,2014-15,Rural Development Projects,Madhya Pradesh,Hoshangabad,Other Implementing Agency,Natraj Proteins Limited,L00153MP1990PLC006090,11.38,0.210000,0.21,0.01,Madhya Pradesh,RoC-Gwalior,Public,Other than Govt.,Company limited by shares,Listed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201044,S Cube Infra,200000.00,200000.0,2018-19,Rural development projects,Telangana,NEC/ Not mentioned,Directly by company,ASIAN INSTITUTE OF GASTROENTEROLOGY PRIVATE LI...,U99999TG1994PTC018352,97.23,1.940000,1.64,1.64,Telangana,RoC-Hyderabad,Private,Other than Govt.,Company limited by Shares,Unlisted
201045,Lake View Sports Foundation,4200000.00,4200000.0,2018-19,Rural development projects,Telangana,NEC/ Not mentioned,Directly by company,ASIAN INSTITUTE OF GASTROENTEROLOGY PRIVATE LI...,U99999TG1994PTC018352,97.23,1.940000,1.64,1.64,Telangana,RoC-Hyderabad,Private,Other than Govt.,Company limited by Shares,Unlisted
201046,Research for promoting health care including p...,12000000.00,12000000.0,2018-19,Rural development projects,Telangana,NEC/ Not mentioned,Other implementing agencies,ASIAN INSTITUTE OF GASTROENTEROLOGY PRIVATE LI...,U99999TG1994PTC018352,97.23,1.940000,1.64,1.64,Telangana,RoC-Hyderabad,Private,Other than Govt.,Company limited by Shares,Unlisted
201047,Contribution to Akshaya Patra,1207000.00,1207000.0,2018-19,"Poverty, Eradicating Hunger, Malnutrition",Telangana,NEC/ Not mentioned,By Trusts/Societies/Section 8 company set up b...,ECIL-RAPISCAN LIMITED,U99999TG1995PLC019129,0.00,0.120000,0.12,0.00,Telangana,RoC-Hyderabad,Public,Other than Govt.,Company limited by shares,Unlisted


In [70]:
master_dataset.year.unique()

array(['2014-15', '2015-16', '2016-17', '2017-18', '2018-19'],
      dtype=object)

In [69]:
master_dataset.to_csv("master_data.csv", index=False)

In [3]:
master_data=pd.read_csv("master_data.csv")


array(['SAB INDUSTRIES LIMITED', 'Natraj Proteins Limited',
       'Usher Agro Limited', ..., 'SUPRAPTI FINVEST PRIVATE LIMITED',
       'LEIGHTON INDIA CONTRACTORS PRIVATE LIMITED',
       'UPS EXPRESS PRIVATE LIMITED'], dtype=object)

In [20]:
master_data[master_data["company name"].str.find("2017")>=0]

,CSR Project(s),Project Amount Outlay,Amount Spent,year,Development Sector(s),State_project,District_project,Mode of Implementation,company name,CIN_Number,Average Net Profit,CSR Prescribed Expenditure,CSR Spent,Local Area Spent,State_company,RoC,Class,Company Type,Sub Category,Listing Status
